In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
import json
import requests
import pickle
from ift6758.client.serving_client import ServingClient

from xgboost import XGBClassifier

In [3]:
# model = pickle.load(open('serving/xgb_best.sav','rb'))

# XGBClassifier()
# model.load_model('xgb_best.sav')

In [4]:
COMET_API_KEY = os.environ.get("COMET_API_KEY")
print(COMET_API_KEY)

None


In [5]:
requests.post("http://0.0.0.0:4000/download_registry_model", 
              json={'workspace': "ift6758-22-team2", 'model': "xgb-best-final", 'version': "1.0.0"})

<Response [200]>

In [6]:
sc = ServingClient(port = 4000)

In [7]:
sc.download_registry_model("ift6758-22-team2", "logreg-with-distance-and-angle", "1.0.0")

'SUCCESS: logreg_dist_angle.sav is loaded!'

In [8]:
df=pd.read_csv('./complete_dataset.csv',index_col=0)

In [9]:
df[['game_period', 'x', 'y', 'distanceNet_or_shotDistance', 'goal', 'shot_angle_absolute', 
    'shotType', 'last_event', 'last_x', 'last_y', 'game_seconds', 'time_from_last_event', 
    'distance_from_last_event', 'rebound', 'speed', 'time_since_powerplay_started', 
    'nbFriendly_non_goalie_skaters', 'nbOpposing_non_goalie_skaters', 'change_in_shot_angle']].head(6)

,game_period,x,y,distanceNet_or_shotDistance,goal,shot_angle_absolute,shotType,last_event,last_x,last_y,game_seconds,time_from_last_event,distance_from_last_event,rebound,speed,time_since_powerplay_started,nbFriendly_non_goalie_skaters,nbOpposing_non_goalie_skaters,change_in_shot_angle
0,1,-35.0,31.0,62.265560,0,29.859016,Slap Shot,Faceoff,-20.0,-22.0,87,12,55.081757,False,4.590146,0,5,5,0.000000
1,1,-78.0,5.0,12.083046,1,24.443955,Tip-In,Faceoff,-69.0,22.0,115,27,19.235384,False,0.712422,0,5,5,0.000000
2,1,-83.0,-20.0,20.880613,0,73.300756,Backhand,Faceoff,0.0,0.0,126,11,85.375641,False,7.761422,0,5,5,0.000000
3,1,48.0,25.0,48.020829,0,31.373005,Wrist Shot,Blocked Shot,73.0,9.0,174,2,29.681644,False,14.840822,0,5,5,0.000000
4,1,-57.0,14.0,34.928498,0,23.629378,Snap Shot,Shot,48.0,25.0,179,5,105.574618,True,21.114924,0,5,5,7.743627
5,1,-49.0,-17.0,43.462628,0,23.025492,Slap Shot,Giveaway,-39.0,-38.0,279,0,23.259407,False,NaN,100,5,4,0.000000


In [10]:
df1 = df.loc[:5].to_json()
pd.read_json(df1)

,index,gameID_eventID,game_period,dateTime,gameID,team,goal,x,y,shooter,...,distance_from_last_event,rebound,speed,time_since_powerplay_started,nbFriendly_non_goalie_skaters,nbOpposing_non_goalie_skaters,team_side,distanceNet_or_shotDistance,shot_angle_absolute,change_in_shot_angle
0,0,201503011155,1,2016-04-15 00:14:04+00:00,2015030111,Florida Panthers,0,-35,31,Alex Petrovic,...,55.081757,False,4.590146,0,5,5,right,62.265560,29.859016,0.000000
1,1,201503011157,1,2016-04-15 00:15:06+00:00,2015030111,Florida Panthers,1,-78,5,Teddy Purcell,...,19.235384,False,0.712422,0,5,5,right,12.083046,24.443955,0.000000
2,2,201503011159,1,2016-04-15 00:16:00+00:00,2015030111,Florida Panthers,0,-83,-20,Nick Bjugstad,...,85.375641,False,7.761422,0,5,5,right,20.880613,73.300756,0.000000
3,3,201503011161,1,2016-04-15 00:16:48+00:00,2015030111,New York Islanders,0,48,25,Thomas Hickey,...,29.681644,False,14.840822,0,5,5,left,48.020829,31.373005,0.000000
4,4,201503011162,1,2016-04-15 00:16:55+00:00,2015030111,Florida Panthers,0,-57,14,Aleksander Barkov,...,105.574618,True,21.114924,0,5,5,right,34.928498,23.629378,7.743627
5,5,201503011167,1,2016-04-15 00:20:30+00:00,2015030111,Florida Panthers,0,-49,-17,Brian Campbell,...,23.259407,False,NaN,100,5,4,right,43.462628,23.025492,0.000000


In [11]:
preds = sc.predict(df)

/Users/subhrajyotidasgupta/Downloads/Mila:UdeM/IFT6758/DS project/nhl-app/utils.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].astype('float')
/Users/subhrajyotidasgupta/Downloads/Mila:UdeM/IFT6758/DS project/nhl-app/utils.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['game_period'] = data['game_period'].astype('str')
COMET INFO: Downloading experiment model 'xgb_preprocess' ...
COMET INFO: Unzipping model to './' ...
COMET INFO: done!


In [13]:
preds

{'0': 0.0304475993,
 '1': 0.1760348702,
 '2': 0.066696667,
 '3': 0.0490369763,
 '4': 0.085930495,
 '5': 0.0649924001}